In [1]:
import pandas as pd
from pandas.tseries.offsets import BDay
import numpy as np
import matplotlib.pyplot as plt
import psycopg2 as pg
from scipy import stats
from cmdstanpy import CmdStanModel
import arviz as az

from config import DATABASE_URI

plt.rcParams["figure.figsize"] = (15,10)

In [2]:
with pg.connect(DATABASE_URI) as conn:
    with conn.cursor() as cur:
        cur.execute("SELECT date, close FROM prices WHERE frequency='MINUTE' AND ticker='^GSPC'")
        result = cur.fetchall()
    
data = pd.DataFrame.from_records(result, columns=["date", "close"], coerce_float=True, index="date").squeeze()

# Daily realized volatility estimator from intraday data
results = {}
for idx, day in data.groupby(data.index.date):
    results[idx] = np.sum(np.square(np.log(day) - np.log(day.shift(1))))
    
vol = pd.Series(results)
vol = vol.reindex(pd.to_datetime(vol.index))

In [3]:
vol

2007-04-30    0.000016
2007-05-01    0.000031
2007-05-02    0.000012
2007-05-03    0.000017
2007-05-04    0.000022
                ...   
2021-12-13    0.000034
2021-12-14    0.000048
2021-12-15    0.000093
2021-12-16    0.000088
2021-12-17    0.000120
Length: 3687, dtype: float64